In [90]:
""" 
pip install pandas
pip install numpy
pip install matplotlib
pip install plotly
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from datetime import date
import plotly.graph_objects as go
from collections import defaultdict
from scipy.optimize import linprog
from scipy.linalg import lstsq
from itertools import product

import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

## **Refinements**


We choose to make 4 refinements to our original implementation. They are listed here in order of siginifgance. 

1. Signal Generation
- We choose to modify our signal such that it more closely aligns with our expected portfolio construction process.

2. Parameter Refinement
- We further choose to refine our parameters of z-scores as we had originally chosen in our first implementation and specification.

3. Portfolio Construction Process
- We refine the portfolio construction process to properly include hedge transactions and proper signal weighted portfolios, which were not properly tested originally due to a lack of the number of signals over our time period.

4. Transaction Costs

- We refine our transaction costs estimates and analyze its impact on the performance of our strategy. 

### **Signal Generation**

Our original signal was formulated as follows:

1. $Spread = Yield_{5yr} - Yield_{7yr} + Yield_{10yr}$ 
2. $Signal = z = \frac{Spread - E[Spread]}{\sigma_{Spread}} $ 
3. Where we estimate the mean and standard deviation over a 100 day rolling window.

We reformulate the signal to more properly incoporate the portfolio construction process being duration neutral.

1. We first begin by constructing a synthetic 7yr yield, $\hat{Y}_{7yr} = w_1 Yield_{5yr} + w_2 Yield_{10yr}$
2. We utilize the duration of these bonds to determine their weight. 

$$
\begin{align}
w_1 & = \frac{d_1}{d_1 +d_2 + d_3} \\
w_2 & = \frac{d_2}{d_1 +d_2 + d_3} 
\end{align}
$$

3. $Spread = \hat{Y}_{7yr} - Yield_{7yr}$
4. $Signal = z = \frac{Spread - E[Spread]}{\sigma_{Spread}} $ 

### **Portfolio Construction**

We implement proper signal weighting for our portfolio and also improved the hedging equation solution which was not correctly specified given the direction we intended to take the trade.

### **Parameter Refinement**

As a result of the change in our signal, and the lack of trades given by our previous signal because of our parameters we change the parameters of the model. We perform a parameter search to best identify the the optimal entry and exit thresholds.

### **Transaction Costs**

We had not previously obtained transaction cost impact analysis. In this refinement we more properly reflect this.

## **Refined Code**

*Position Class*


Note that we do not make any notable changes to the position class. We simply use this for easier state tracking and portfolio calculations.

In [29]:
class Position:

    def __init__(self, entry_date, bond, entry_price, quantity,
                 direction, duration, trade_id, maturities,is_hedge = False):
        
        self.entry_date = entry_date
        self.bond = bond
        self.entry_price = entry_price  
        self.quantity = quantity 
        self.direction = direction 
        self.duration = duration  
        self.trade_id = trade_id  
        self.maturities = maturities  # List of maturities associated with this trade
        self.is_hedge = is_hedge
        self.closed = False
        self.exit_date = None
        self.exit_price = None

    def close(self,exit_date,exit_price):
        """ 
        Closes a position
        """
        
        self.exit_date = exit_date
        self.exit_price = exit_price
        self.closed = True
    def __str__(self):
        status = "Closed" if self.closed else "Open"
        hedge_status = "Hedge" if self.is_hedge else "Non-Hedge"
        exit_details = (
            f", Exit Date: {self.exit_date}, Exit Price: {self.exit_price}"
            if self.closed else ""
        )
        return (f"Position({status}) - Trade ID: {self.trade_id}, "
                f"Bond: {self.bond}, Entry Date: {self.entry_date}, Entry Price: {self.entry_price}, "
                f"Quantity: {self.quantity}, Direction: {self.direction}, "
                f"Duration: {self.duration}, Maturities: {self.maturities}, {hedge_status}"
                f"{exit_details}")

*Strategy Class*

I include 3 flags. 
1. SIGNAL_FLAG = False if using old signal, True if using new signal
2. PORT_CONSTRUCT_FLAG = False is using old portfolio construct, True if using new
3. TXC_FLAG = False if not applying txc (old), True if applying TXC to each trade

In [ ]:
class YCA:
    def __init__(self, data, start_date, end_date, window_length, z_enter, z_exit, universe, txc=0.0005,hedge_threshold=0.5):
        """
        Initialize the strategy.
        """
        self.data = data
        self.start_date = start_date
        self.end_date = end_date
        self.window_length = window_length
        self.z_enter = z_enter
        self.z_exit = z_exit
        self.universe = universe
        self.txc = txc
        self.hedge_threshold = hedge_threshold

        # State tracking
        self.signals = defaultdict(list)
        self.positions = []
        self.daily_returns = pd.Series(dtype='float64')
        self.daily_duration = pd.Series(dtype='float64')
        self.daily_dollar_exposure = pd.Series(dtype='float64')
        self.daily_value = pd.Series(dtype='float64')

        self.SIGNAL_FLAG = False
        self.PORT_CONSTRUCT_FLAG = False
        self.TXC_FLAG = False
        self.debug = False

    def run(self):
        """Run the strategy over the specified date range."""


        for idx in range(self.window_length, len(self.data)):
            dt = self.data.index[idx]
            #print(dt)
            # 1. Generate Signals
            self.get_signals(idx)

            # 2. Close Positions
            self.close_positions(idx)

            # 3. Portfolio Construction
            self.construct_portfolio(idx)

            # 4. Compute Statistics
            self.log_portfolio_stats(idx)
        
        self.clean_stats()



    def get_signals(self,idx):
        if self.SIGNAL_FLAG:
            self._signal_refinement(idx)
        else:
            self._signal_original(idx)
    
    def _signal_original(self,idx):
        """Original Signal Generation"""
        dt = self.data.index[idx]

        for maturities in self.universe:

            low, mid, high = maturities

            # Observed
            x1, x2, x3 = self.data.loc[(dt, 'TDYTM')][low], self.data.loc[(dt, 'TDYTM')][mid],self.data.loc[(dt, 'TDYTM')][high]

            # Window
            y1, y2, y3 = [self.data.iloc[idx - self.window_length: idx, :]['TDYTM'][maturity].T.values for maturity in [low, mid, high]]

            # Spread
            obs_spread = x1 - x2 + x3
            window_spread = y1 - y2 + y3

            # Z-score
            z = (obs_spread - window_spread.mean()) / window_spread.std()

            # Log z-scores
            if abs(z) > self.z_enter:
                self.signals[dt].append([z,maturities])

    def _signal_refinement(self,idx):
        """Refined Signal Generation"""
        dt = self.data.index[idx]

        for maturities in self.universe:
            low, mid, high = maturities

            # Observed
            yields_obs = self.data.iloc[idx, :]['TDYTM'][maturities]
            durations_obs = self.data.iloc[idx, :]['TDDURATN'][[low,high]]
            weights_obs = durations_obs / durations_obs.sum()
            synthetic_mid_obs = weights_obs[low]*yields_obs[low] + weights_obs[high]*yields_obs[high]

            # Window
            yields = self.data.iloc[idx - self.window_length: idx, :]['TDYTM'][maturities]
            durations = self.data.iloc[idx - self.window_length: idx, :]['TDDURATN'][[low,high]]
            weights = durations.div(durations.sum(axis = 1),axis = 0)
            synthetic_mid = weights[low]*yields[low] + weights[high]*yields[high]

            # Spread - NEED TO REMOVE THE NEGATIVE
            spread_obs = -(synthetic_mid_obs - yields_obs[mid])
            spread_window = -(synthetic_mid - yields[mid])
            
            # Z-Score
            z = (spread_obs - spread_window.mean()) / spread_window.std()
            
            # Log z-scores
            if abs(z) > self.z_enter:
                self.signals[dt].append([z,maturities])
    
    def close_positions(self,idx):
        if self.SIGNAL_FLAG:
            self._close_refinement(idx)
        else:
            self._close_original(idx)

    def _close_original(self, idx):
        dt = self.data.index[idx]
        trades_to_close = set()

        for pos in self.positions:
            # only close open, traded positions
            if not pos.closed and not pos.is_hedge: 
                low, mid, high = pos.maturities

                # Observed
                x1, x2, x3 = self.data.loc[(dt, 'TDYTM')][low], self.data.loc[(dt, 'TDYTM')][mid],self.data.loc[(dt, 'TDYTM')][high]

                # Window
                y1, y2, y3 = [self.data.iloc[idx - self.window_length: idx, :]['TDYTM'][maturity].T.values for maturity in [low, mid, high]]

                # Spread
                obs_spread = x1 - x2 + x3
                window_spread = y1 - y2 + y3

                # Z-score
                z = (obs_spread - window_spread.mean()) / window_spread.std()

                # Close Reverted Trades
                if abs(z) < self.z_exit:
                    trades_to_close.add(pos)
        
        # Close Trades
        for pos in trades_to_close:
            if not pos.closed:
                exit_price = self.data.loc[(dt, 'TDNOMPRC')][pos.bond]
                pos.close(exit_date = dt,exit_price = exit_price)

    def _close_refinement(self, idx):
        """Refinement of Close Positions to Account for New Signal"""
        dt = self.data.index[idx]
        trades_to_close = set()

        for pos in self.positions:           
            # only close open, traded positions
            if not pos.closed and not pos.is_hedge: 
                low, mid, high = pos.maturities

                # Observed
                yields_obs = self.data.iloc[idx, :]['TDYTM'][pos.maturities]
                durations_obs = self.data.iloc[idx, :]['TDDURATN'][[low,high]]
                weights_obs = durations_obs / durations_obs.sum()
                synthetic_mid_obs = weights_obs[low]*yields_obs[low] + weights_obs[high]*yields_obs[high]

                # Window
                yields = self.data.iloc[idx - self.window_length: idx, :]['TDYTM'][pos.maturities]
                durations = self.data.iloc[idx - self.window_length: idx, :]['TDDURATN'][[low,high]]
                weights = durations.div(durations.sum(axis = 1),axis = 0)
                synthetic_mid = weights[low]*yields[low] + weights[high]*yields[high]

                # Spread - Should be Changed
                spread_obs = -(synthetic_mid_obs - yields_obs[mid])
                spread_window = -(synthetic_mid - yields[mid])
                
                # Z-Score
                z = (spread_obs - spread_window.mean()) / spread_window.std()
                
                # Close Reverted Trades
                if abs(z) < self.z_exit:
                    trades_to_close.add(pos)

        # Close Trades
        for pos in trades_to_close:
            if not pos.closed:
                exit_price = self.data.loc[(dt, 'TDNOMPRC')][pos.bond]
                pos.close(exit_date = dt,exit_price = exit_price)

    def construct_portfolio(self,idx):
        """Handle Portfolio Construction Refinement"""
        if self.PORT_CONSTRUCT_FLAG:
            self._construct_portfolio_refinement(idx)
        else:
            self._construct_portfolio_original(idx)

    def _construct_portfolio_original(self, idx):
        """Original Portfolio Construction"""
        dt = self.data.index[idx] 

        # If no signal for the current date, skip
        if dt not in self.signals:
            return  

        for zscore, maturities in self.signals[dt]:
            low, mid, high = maturities 
            trade_id = f"trade_{dt}_{low}_{mid}_{high}"

            # Get prices for the bonds
            p_s = self.data.loc[(dt, 'TDNOMPRC')][low]
            p_m = self.data.loc[(dt, 'TDNOMPRC')][mid]
            p_l = self.data.loc[(dt, 'TDNOMPRC')][high]

            # Get durations for the bonds
            d_s = self.data.loc[(dt, 'TDDURATN')][low]
            d_m = self.data.loc[(dt, 'TDDURATN')][mid]
            d_l = self.data.loc[(dt, 'TDDURATN')][high]

            # Amount allocated to middle bond
            q_m = 100 

            # Solve for quantities based on dollar-duration neutrality
            q_s, q_l = self._solve_dollar_duration_neutral(p_s, p_m, p_l, d_s, d_m, d_l, q_m)

            # Place Orders
            if zscore < 0:  # Yield Curve Butterfly Trough [Short Body, Long Wings]
                self._place_order(dt, low, p_s, int(q_s), 'long', d_s, trade_id, maturities,is_hedge=False, z=zscore)
                self._place_order(dt, mid, p_m, int(q_m), 'short', d_m, trade_id, maturities, is_hedge=False, z=zscore)
                self._place_order(dt, high, p_l, int(q_l), 'long', d_l, trade_id, maturities, is_hedge=False, z=zscore)

            if zscore > 0:  # Yield Curve Butterfly Hump [Long Body, Short Wings]
                self._place_order(dt, low, p_s, int(q_s), 'short', d_s, trade_id, maturities, is_hedge=False, z=zscore)
                self._place_order(dt, mid, p_m, int(q_m), 'long', d_m, trade_id, maturities, is_hedge=False, z=zscore)
                self._place_order(dt, high, p_l, int(q_l), 'short', d_l, trade_id, maturities, is_hedge=False, z=zscore)
    
    def _solve_dollar_duration_neutral(self,P_s, P_m, P_l, D_s, D_m, D_l, q_m):

        """ 
        Solves the dollar-duration neutral required positions given inputs.

        A [q_s,q_l]       = b  
        p_s*q_s + p_l*q_l = q_m*p_m
        d_s*q_s + d_l*q_l = q_m*d_m
        """

        A = np.array([
            [P_s, P_l],
            [D_s, D_l]
        ])
        
        b = np.array([
            q_m * P_m,
            q_m * D_m
        ])
        
        # Solve
        q_s, q_l = np.linalg.solve(A, b)
        
        return q_s, q_l 

    def _construct_portfolio_refinement(self, idx):
        dt = self.data.index[idx]

        # No Signal Generated
        if dt not in self.signals:
            return
        
        for zscore, maturities in self.signals[dt]:
            low, mid, high = maturities 
            trade_id = f"trade_{dt}_{low}_{mid}_{high}"

            # Get prices for the bonds
            p_s = self.data.loc[(dt, 'TDNOMPRC')][low]
            p_m = self.data.loc[(dt, 'TDNOMPRC')][mid]
            p_l = self.data.loc[(dt, 'TDNOMPRC')][high]

            # Get durations for the bonds
            d_s = self.data.loc[(dt, 'TDDURATN')][low]
            d_m = self.data.loc[(dt, 'TDDURATN')][mid]
            d_l = self.data.loc[(dt, 'TDDURATN')][high]
    
            # Amount allocated to middle bond
            if zscore < 0:
                q_m = -100*abs(zscore)/sum([abs(z) for z,_ in self.signals[dt]])
            else:
                q_m = 100*abs(zscore)/sum([abs(z) for z,_ in self.signals[dt]])

            duration = q_m*d_m
            value = q_m*p_m

            a = p_s
            b = p_l

            c = d_s
            d = d_l

            q_s = (b * duration - value * d) / (a * d - b * c)
            q_l = (value * c - a * duration) / (a * d - b * c)
            s_d = 'short' if q_s < 0 else 'long'
            l_d = 'short' if q_l < 0 else 'long'
            
            
            if self.debug:
                print(f"Date: {dt}, Dollar Value:{q_s*p_s + q_m*p_m + q_l*p_l}, Duration:{q_s*d_s + q_m*d_m + q_l*d_l}")
            # Place Orders
            if zscore < 0:  # Yield Curve Butterfly Trough [Short Body, Long Wings]
                self._place_order(dt, low, p_s, int(abs(q_s)),s_d, d_s, trade_id, maturities,is_hedge=False,z = zscore)
                self._place_order(dt, mid, p_m, int(abs(q_m)), 'short', d_m, trade_id, maturities, is_hedge=False,z= zscore)
                self._place_order(dt, high, p_l, int(abs(q_l)),l_d, d_l, trade_id, maturities, is_hedge=False,z = zscore)

            if zscore > 0:  # Yield Curve Butterfly Hump [Long Body, Short Wings]
                self._place_order(dt, low, p_s, int(abs(q_s)),s_d, d_s, trade_id, maturities, is_hedge=False,z = zscore)
                self._place_order(dt, mid, p_m, int(abs(q_m)),'long', d_m, trade_id, maturities, is_hedge=False,z = zscore)
                self._place_order(dt, high, p_l, int(abs(q_l)),l_d, d_l, trade_id, maturities, is_hedge=False,z = zscore)

    def _place_order(self, dt, bond, price, qty, direction, duration,trade_id, maturities,z, is_hedge=False):
        """Helper to create and store a new position."""
        if direction == 'long':
            price = price * (1 + self.txc)
        else:
            price = price * (1 - self.txc)
        pos = Position(
            entry_date=dt, bond=bond, entry_price=price, duration=duration,
            quantity=qty, direction=direction, trade_id=trade_id,
            maturities=maturities, is_hedge=is_hedge
        )
        pos.z = z
        self.positions.append(pos)

    def log_portfolio_stats(self,idx):
        """Compute and store all portfolio stats for given day"""

        dt = self.data.index[idx]
        daily_returns, portfolio_value = self._calculate_daily_returns(idx)
        daily_duration, portfolio_duration = self._calculate_daily_duration(idx)
        
        self.daily_returns[dt] = daily_returns
        self.daily_duration[dt] = daily_duration
        self.daily_value[dt] = portfolio_value

    def _calculate_daily_returns(self,idx):
        """Calculate Portfolio Returns"""

        dt = self.data.index[idx]
        total_value = 0
        weighted_daily_return = 0
        daily_net_balance = 0
        for pos in self.positions:
            if not pos.closed:
                # Determine the last price
                entry_idx = self.data.index.get_loc(pos.entry_date)
                last_price = pos.entry_price if idx == entry_idx + 1 else self.data.iloc[idx - 1]['TDNOMPRC'][pos.bond]

                # Current price
                curr_price = self.data.iloc[idx]['TDNOMPRC'][pos.bond]

                # Calculate return and weighted contribution
                position_value = last_price * pos.quantity
                daily_net_balance += last_price*pos.quantity * (1 if pos.direction == 'long' else -1)
                position_return = (curr_price / last_price - 1) * (1 if pos.direction == 'long' else -1)

                total_value += position_value
                weighted_daily_return += position_value * position_return
        
        portfolio_return = weighted_daily_return / total_value if total_value != 0 else 0
        return portfolio_return,daily_net_balance

    def _calculate_daily_duration(self,idx):
        """Calculate Portfolio Duration"""
        total_quantity_duration = 0
        total_quantity = 0

        for position in self.positions:
            if not position.closed:

                # Calculate the effective duration for the position
                effective_duration = position.duration * position.quantity * (1 if position.direction == 'long' else -1)

                # Update total quantity-duration and total quantity
                total_quantity_duration += effective_duration
                total_quantity += position.quantity

        # Avoid division by zero
        if total_quantity == 0:
            return 0, 0

        # Calculate daily duration in units (quantity * duration)
        daily_duration = total_quantity_duration / total_quantity 

        # Calculate portfolio duration exposure (in units)
        portfolio_duration_exposure = daily_duration * total_quantity

        return daily_duration, portfolio_duration_exposure
    
    def clean_stats(self):
        """Clean Return Series + Compute Stats"""
        _df = pd.DataFrame(self.daily_returns, columns=['daily_return'])
        _df['cum_return'] = (1+_df['daily_return']).cumprod()

        self._clean_returns = _df

        _df = pd.DataFrame(self.daily_duration,columns=['duration'])
        self._clean_duration = _df

## **Analysis**

*Data*

In [112]:
# Yields/Price/Duration
df = pd.read_csv('CRSP_UST.csv')

# Filter
df = df[df['TIDXFAM'] == 'FIXEDTERM'][['CALDT','TTERMLBL','TDNOMPRC','TDYTM','TDDURATN']]

# Multi Index
df = (df
 .sort_values(['CALDT','TTERMLBL'],ascending=True)
 .pivot(index = 'CALDT',columns='TTERMLBL',values = ['TDNOMPRC','TDYTM','TDDURATN'])
 )

new_column_names = {
    'CRSP Fixed Term Index - 1-Year (Nominal)': '1yr', 'CRSP Fixed Term Index - 2-Year (Nominal)': '2yr',
    'CRSP Fixed Term Index - 5-Year (Nominal)': '5yr', 'CRSP Fixed Term Index - 7-Year (Nominal)': '7yr',
    'CRSP Fixed Term Index - 10-Year (Nominal)': '10yr', 'CRSP Fixed Term Index - 20-Year (Nominal)': '20yr',
    'CRSP Fixed Term Index - 30-Year (Nominal)': '30yr'
}

# Rename columns
df.columns = pd.MultiIndex.from_tuples(
    [(level_0, new_column_names.get(level_1, level_1)) for level_0, level_1 in df.columns],
    names=df.columns.names
)

# Reorder columns
column_order = ['1yr', '2yr', '5yr', '7yr', '10yr', '20yr', '30yr']
ordered_columns = pd.MultiIndex.from_tuples(
    [(level_0, col) for level_0 in df.columns.get_level_values(0).unique() for col in column_order],
    names=df.columns.names
)

df = df[ordered_columns]

In [140]:
# In-Sample Dates
start_date = date(1990,1,1)
end_date = date(2010,1,1)

# Esimtation Window Length for z-scores
window_length = 100

# z-score threshold
z_enter = 3
z_exit = 1.25

# Traded Maturities (Triplet of Low, Mid, High)
universe = [['1yr','2yr','5yr'],['2yr','5yr','7yr'],['5yr','7yr','10yr'],['7yr','10yr','20yr'],['10yr','20yr','30yr']]
#universe = [['5yr','7yr','10yr']]

# txc
txc = 0.005 # 0.0005

# Hedge Threshold
hedge_threshold = 0.5

# Filter for In-Sample Data
df.index = pd.to_datetime(df.index).date
df = df[(df.index <= end_date) & (df.index >= start_date)]

### **Results of Improvements**

#### **Improvements Comparison**

In [141]:
"""
u1 = [['1yr','2yr','5yr'],['2yr','5yr','7yr'],['5yr','7yr','10yr'],['7yr','10yr','20yr'],['10yr','20yr','30yr']]
u2 = [['5yr','7yr','10yr']]

param_grid = product(
    [True, False],                              # SIGNAL_FLAG
    [(True, 3, 1.25), (False, 4, 1)],           # (PARAM_FLAG, enter, exit)
    [(False, 0), (True, 0.005)],                # (TXC_FLAG, txc)
    [True, False],                              # PORTFOLIO_FLAG
    [u1, u2]                                    # universe
)

results = []

i = 0
for signal_flag, (param_flag, enter, exit), (txc_flag, txc), portfolio_flag, universe in param_grid:
    print(i)
    s = YCA(data=df, start_date=start_date,end_date=end_date,window_length=window_length,
        z_enter=enter,z_exit=exit,universe=universe,txc=txc,hedge_threshold=hedge_threshold)
    
    s.SIGNAL_FLAG = signal_flag
    s.TXC_FLAG = txc_flag
    s.PORT_CONSTRUCT_FLAG = portfolio_flag

    s.run()
    returns = s._clean_returns
    sr = returns['daily_return'].mean()/returns['daily_return'].std() * np.sqrt(252)
    running_max = returns['cum_return'].cummax()
    drawdown = (returns['cum_return'] - running_max) / running_max  
    max_drawdown = drawdown.min()*100
    mu = returns['daily_return'].mean()
    dsd = returns[returns['daily_return'] <= 0]['daily_return'].std()
    sor = mu/dsd * np.sqrt(252)
    mean_ret = returns['daily_return'].mean()*100*252
    std = returns['daily_return'].std()*100*np.sqrt(252)
    results.append({
        'signal_flag': signal_flag,
        'param_flag': param_flag,
        'entry_threshold': enter,
        'exit_threshold': exit,
        'txc_flag': txc_flag,
        'transaction_cost': txc,
        'portfolio_flag': portfolio_flag,
        'universe': 'u1' if universe == u1 else 'u2',
        'num_signals': sum(len(t) for t in s.signals.values()),
        'Annualized Ret':mean_ret,
        'Annualized Vol':std,
        'Sharpe Ratio':sr,
        'Sortino Ratio':sor,
        'Maximum Drawdown':max_drawdown,
    })
    i += 1

results_df = pd.DataFrame(results) 
results_df.sort_values(by = 'Sharpe Ratio',ascending=False)
"""

"\nu1 = [['1yr','2yr','5yr'],['2yr','5yr','7yr'],['5yr','7yr','10yr'],['7yr','10yr','20yr'],['10yr','20yr','30yr']]\nu2 = [['5yr','7yr','10yr']]\n\nparam_grid = product(\n    [True, False],                              # SIGNAL_FLAG\n    [(True, 3, 1.25), (False, 4, 1)],           # (PARAM_FLAG, enter, exit)\n    [(False, 0), (True, 0.005)],                # (TXC_FLAG, txc)\n    [True, False],                              # PORTFOLIO_FLAG\n    [u1, u2]                                    # universe\n)\n\nresults = []\n\ni = 0\nfor signal_flag, (param_flag, enter, exit), (txc_flag, txc), portfolio_flag, universe in param_grid:\n    print(i)\n    s = YCA(data=df, start_date=start_date,end_date=end_date,window_length=window_length,\n        z_enter=enter,z_exit=exit,universe=universe,txc=txc,hedge_threshold=hedge_threshold)\n    \n    s.SIGNAL_FLAG = signal_flag\n    s.TXC_FLAG = txc_flag\n    s.PORT_CONSTRUCT_FLAG = portfolio_flag\n\n    s.run()\n    returns = s._clean_returns\n    sr = r

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>signal_flag</th>
      <th>param_flag</th>
      <th>entry_threshold</th>
      <th>exit_threshold</th>
      <th>txc_flag</th>
      <th>transaction_cost</th>
      <th>portfolio_flag</th>
      <th>universe</th>
      <th>num_signals</th>
      <th>Annualized Ret</th>
      <th>Annualized Vol</th>
      <th>Sharpe Ratio</th>
      <th>Sortino Ratio</th>
      <th>Maximum Drawdown</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>True</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>False</td>
      <td>0.000</td>
      <td>True</td>
      <td>u1</td>
      <td>532</td>
      <td>11.331241</td>
      <td>13.373530</td>
      <td>0.847289</td>
      <td>3.878508</td>
      <td>-10.045087</td>
    </tr>
    <tr>
      <th>2</th>
      <td>True</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>False</td>
      <td>0.000</td>
      <td>False</td>
      <td>u1</td>
      <td>532</td>
      <td>15.495100</td>
      <td>19.617317</td>
      <td>0.789868</td>
      <td>5.063893</td>
      <td>-10.535958</td>
    </tr>
    <tr>
      <th>1</th>
      <td>True</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>False</td>
      <td>0.000</td>
      <td>True</td>
      <td>u2</td>
      <td>110</td>
      <td>5.487694</td>
      <td>8.676859</td>
      <td>0.632452</td>
      <td>21.606825</td>
      <td>-0.785146</td>
    </tr>
    <tr>
      <th>6</th>
      <td>True</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>True</td>
      <td>0.005</td>
      <td>False</td>
      <td>u1</td>
      <td>532</td>
      <td>12.111259</td>
      <td>19.640912</td>
      <td>0.616634</td>
      <td>3.869238</td>
      <td>-28.123903</td>
    </tr>
    <tr>
      <th>4</th>
      <td>True</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>True</td>
      <td>0.005</td>
      <td>True</td>
      <td>u1</td>
      <td>532</td>
      <td>7.927421</td>
      <td>13.402572</td>
      <td>0.591485</td>
      <td>2.652746</td>
      <td>-28.617546</td>
    </tr>
    <tr>
      <th>3</th>
      <td>True</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>False</td>
      <td>0.000</td>
      <td>False</td>
      <td>u2</td>
      <td>110</td>
      <td>11.383037</td>
      <td>21.768631</td>
      <td>0.522910</td>
      <td>10.199674</td>
      <td>-5.707112</td>
    </tr>
    <tr>
      <th>5</th>
      <td>True</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>True</td>
      <td>0.005</td>
      <td>True</td>
      <td>u2</td>
      <td>110</td>
      <td>4.515118</td>
      <td>8.685910</td>
      <td>0.519821</td>
      <td>8.155263</td>
      <td>-4.012794</td>
    </tr>
    <tr>
      <th>7</th>
      <td>True</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>True</td>
      <td>0.005</td>
      <td>False</td>
      <td>u2</td>
      <td>110</td>
      <td>10.427700</td>
      <td>21.772367</td>
      <td>0.478942</td>
      <td>8.360870</td>
      <td>-5.707112</td>
    </tr>
    <tr>
      <th>9</th>
      <td>True</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>False</td>
      <td>0.000</td>
      <td>True</td>
      <td>u2</td>
      <td>32</td>
      <td>3.094808</td>
      <td>7.212519</td>
      <td>0.429088</td>
      <td>15.927769</td>
      <td>-0.875749</td>
    </tr>
    <tr>
      <th>10</th>
      <td>True</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>False</td>
      <td>0.000</td>
      <td>False</td>
      <td>u1</td>
      <td>136</td>
      <td>10.310092</td>
      <td>24.885316</td>
      <td>0.414304</td>
      <td>1.617589</td>
      <td>-26.439705</td>
    </tr>
    <tr>
      <th>11</th>
      <td>True</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>False</td>
      <td>0.000</td>
      <td>False</td>
      <td>u2</td>
      <td>32</td>
      <td>9.723350</td>
      <td>24.288423</td>
      <td>0.400329</td>
      <td>8.953113</td>
      <td>-5.864286</td>
    </tr>
    <tr>
      <th>15</th>
      <td>True</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>True</td>
      <td>0.005</td>
      <td>False</td>
      <td>u2</td>
      <td>32</td>
      <td>9.406329</td>
      <td>24.291283</td>
      <td>0.387231</td>
      <td>8.081051</td>
      <td>-5.864286</td>
    </tr>
    <tr>
      <th>13</th>
      <td>True</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>True</td>
      <td>0.005</td>
      <td>True</td>
      <td>u2</td>
      <td>32</td>
      <td>2.765633</td>
      <td>7.219433</td>
      <td>0.383082</td>
      <td>7.422197</td>
      <td>-2.475907</td>
    </tr>
    <tr>
      <th>14</th>
      <td>True</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>True</td>
      <td>0.005</td>
      <td>False</td>
      <td>u1</td>
      <td>136</td>
      <td>9.132845</td>
      <td>24.893463</td>
      <td>0.366877</td>
      <td>1.434661</td>
      <td>-26.756877</td>
    </tr>
    <tr>
      <th>8</th>
      <td>True</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>False</td>
      <td>0.000</td>
      <td>True</td>
      <td>u1</td>
      <td>136</td>
      <td>4.304056</td>
      <td>11.964038</td>
      <td>0.359749</td>
      <td>0.687712</td>
      <td>-24.342817</td>
    </tr>
    <tr>
      <th>25</th>
      <td>False</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>False</td>
      <td>0.000</td>
      <td>True</td>
      <td>u2</td>
      <td>3</td>
      <td>1.240324</td>
      <td>3.688442</td>
      <td>0.336273</td>
      <td>7.551828</td>
      <td>-0.793209</td>
    </tr>
    <tr>
      <th>27</th>
      <td>False</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>False</td>
      <td>0.000</td>
      <td>False</td>
      <td>u2</td>
      <td>3</td>
      <td>1.520492</td>
      <td>4.586291</td>
      <td>0.331530</td>
      <td>7.099563</td>
      <td>-1.127236</td>
    </tr>
    <tr>
      <th>29</th>
      <td>False</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>True</td>
      <td>0.005</td>
      <td>True</td>
      <td>u2</td>
      <td>3</td>
      <td>1.163132</td>
      <td>3.696649</td>
      <td>0.314645</td>
      <td>3.936596</td>
      <td>-1.290645</td>
    </tr>
    <tr>
      <th>31</th>
      <td>False</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>True</td>
      <td>0.005</td>
      <td>False</td>
      <td>u2</td>
      <td>3</td>
      <td>1.443299</td>
      <td>4.593804</td>
      <td>0.314184</td>
      <td>4.266444</td>
      <td>-1.623596</td>
    </tr>
    <tr>
      <th>17</th>
      <td>False</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>False</td>
      <td>0.000</td>
      <td>True</td>
      <td>u2</td>
      <td>73</td>
      <td>1.803591</td>
      <td>6.624804</td>
      <td>0.272248</td>
      <td>0.543845</td>
      <td>-17.067406</td>
    </tr>
    <tr>
      <th>12</th>
      <td>True</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>True</td>
      <td>0.005</td>
      <td>True</td>
      <td>u1</td>
      <td>136</td>
      <td>3.103868</td>
      <td>11.977071</td>
      <td>0.259151</td>
      <td>0.497033</td>
      <td>-26.685768</td>
    </tr>
    <tr>
      <th>19</th>
      <td>False</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>False</td>
      <td>0.000</td>
      <td>False</td>
      <td>u2</td>
      <td>73</td>
      <td>3.404937</td>
      <td>14.328660</td>
      <td>0.237631</td>
      <td>0.666617</td>
      <td>-25.100232</td>
    </tr>
    <tr>
      <th>23</th>
      <td>False</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>True</td>
      <td>0.005</td>
      <td>False</td>
      <td>u2</td>
      <td>73</td>
      <td>2.642053</td>
      <td>14.338792</td>
      <td>0.184259</td>
      <td>0.515891</td>
      <td>-27.264001</td>
    </tr>
    <tr>
      <th>21</th>
      <td>False</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>True</td>
      <td>0.005</td>
      <td>True</td>
      <td>u2</td>
      <td>73</td>
      <td>1.042536</td>
      <td>6.645065</td>
      <td>0.156889</td>
      <td>0.311334</td>
      <td>-19.507934</td>
    </tr>
    <tr>
      <th>26</th>
      <td>False</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>False</td>
      <td>0.000</td>
      <td>False</td>
      <td>u1</td>
      <td>31</td>
      <td>0.265472</td>
      <td>2.823562</td>
      <td>0.094020</td>
      <td>0.176266</td>
      <td>-8.209029</td>
    </tr>
    <tr>
      <th>18</th>
      <td>False</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>False</td>
      <td>0.000</td>
      <td>False</td>
      <td>u1</td>
      <td>429</td>
      <td>0.962555</td>
      <td>15.701522</td>
      <td>0.061303</td>
      <td>0.114311</td>
      <td>-45.088183</td>
    </tr>
    <tr>
      <th>24</th>
      <td>False</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>False</td>
      <td>0.000</td>
      <td>True</td>
      <td>u1</td>
      <td>31</td>
      <td>0.099037</td>
      <td>2.676848</td>
      <td>0.036998</td>
      <td>0.056237</td>
      <td>-9.636291</td>
    </tr>
    <tr>
      <th>30</th>
      <td>False</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>True</td>
      <td>0.005</td>
      <td>False</td>
      <td>u1</td>
      <td>31</td>
      <td>-0.050856</td>
      <td>2.850048</td>
      <td>-0.017844</td>
      <td>-0.032699</td>
      <td>-10.710635</td>
    </tr>
    <tr>
      <th>22</th>
      <td>False</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>True</td>
      <td>0.005</td>
      <td>False</td>
      <td>u1</td>
      <td>429</td>
      <td>-0.681147</td>
      <td>15.723754</td>
      <td>-0.043320</td>
      <td>-0.081254</td>
      <td>-49.021992</td>
    </tr>
    <tr>
      <th>28</th>
      <td>False</td>
      <td>False</td>
      <td>4</td>
      <td>1.00</td>
      <td>True</td>
      <td>0.005</td>
      <td>True</td>
      <td>u1</td>
      <td>31</td>
      <td>-0.223256</td>
      <td>2.703502</td>
      <td>-0.082580</td>
      <td>-0.123958</td>
      <td>-12.074665</td>
    </tr>
    <tr>
      <th>16</th>
      <td>False</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>False</td>
      <td>0.000</td>
      <td>True</td>
      <td>u1</td>
      <td>429</td>
      <td>-0.834411</td>
      <td>9.853337</td>
      <td>-0.084683</td>
      <td>-0.100981</td>
      <td>-43.925720</td>
    </tr>
    <tr>
      <th>20</th>
      <td>False</td>
      <td>True</td>
      <td>3</td>
      <td>1.25</td>
      <td>True</td>
      <td>0.005</td>
      <td>True</td>
      <td>u1</td>
      <td>429</td>
      <td>-2.464935</td>
      <td>9.886560</td>
      <td>-0.249322</td>
      <td>-0.299557</td>
      <td>-47.739132</td>
    </tr>
  </tbody>
</table>
</div>

#### **Results with All Improvements**

In [142]:
s = YCA(data = df, 
            start_date = start_date, end_date = end_date,
            window_length= window_length, z_enter = z_enter,z_exit = z_exit,universe = universe,
            txc = txc, hedge_threshold = hedge_threshold)
s.SIGNAL_FLAG = True
s.PORT_CONSTRUCT_FLAG = True
s.TXC_FLAG = True
s.run()

In [143]:
returns = s._clean_returns

*Cumulative Return*

In [144]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = returns.index,
        y = returns['cum_return'],
    )
)

fig.update_layout(title = 'Cumulative Returns (In Sample)')
fig.update_xaxes(title= 'Date')
fig.update_yaxes(title = 'Cumulative Return')
fig.show()

*Daily Returns*

In [145]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = returns.index,
        y = returns['daily_return'],
    )
)

fig.update_layout(title = 'Daily Returns (In Sample)')
fig.update_xaxes(title= 'Date')
fig.update_yaxes(title = 'Return')
fig.show()

*Hedge Effectiveness (Duration)*

In [146]:
d = s._clean_duration

In [147]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = d.index,
        y = d['duration'],
    )
)

fig.update_layout(title = 'Duration Exposure (In Sample)')
fig.update_xaxes(title= 'Date')
fig.update_yaxes(title = 'Duration (Days)')
fig.show()

### 4. Statistics

**Sharpe Ratio**

In [148]:
print(f'Annualize Sharpe Ratio {returns['daily_return'].mean()/returns['daily_return'].std() * np.sqrt(252)}')

Annualize Sharpe Ratio 0.5914850077804993


**Return Statistics**

In [149]:
print(f'Mean Returns: {returns['daily_return'].mean()*100*252}')
print(f'Standard Deviation of Returns: {returns['daily_return'].std()*100*np.sqrt(252)}')
print(f'Return Skewness {skew(returns['daily_return'])}') # low sample points
print(f'Return Kurtosis {kurtosis(returns['daily_return'])}') # low sample points

Mean Returns: 7.927420532660746
Standard Deviation of Returns: 13.402572218030961
Return Skewness 17.669175832846634
Return Kurtosis 369.470973695961


**Drawdown**

In [150]:
running_max = returns['cum_return'].cummax()
drawdown = (returns['cum_return'] - running_max) / running_max  
max_drawdown = drawdown.min() 
print(f'Maximum Drawdown: {max_drawdown*100}')

Maximum Drawdown: -28.61754618468527


**Sortino Ratio**

In [151]:
mu = returns['daily_return'].mean()
dsd = returns[returns['daily_return'] <= 0]['daily_return'].std()
print(f'Annualized Sortino Ratio {mu/dsd * np.sqrt(252)}') # not many negative returns

Annualized Sortino Ratio 2.6527463464965884


**Number of Trades Placed**


In [152]:
uid = set()
for p in s.positions:
    if not p.is_hedge:
        uid.add(p.trade_id)
print(f'Number of Positions Taken: {len(uid)}')

Number of Positions Taken: 532


**Average Time in Market**

In [153]:
market_days = set()
for p in s.positions:
    if not p.is_hedge:
        market_days.add((p.exit_date - p.entry_date).days)
print(f'Average Time in Market per Trade {round(np.mean(list(market_days)),2)}')

Average Time in Market per Trade 45.61
